# Скачиваем полезное

In [1]:
! pip install razdel

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
! pip install pymorphy2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [3]:
! pip3 install torch torchvision torchaudio

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [4]:
! pip3 install transformers

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [5]:
! pip install evaluate

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [8]:
! pip install transformers datasets faiss-cpu psutil

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [ ]:
! git clone https://github.com/AIRI-Institute/Probing_framework

In [1]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [2]:
import gc
import os
import csv
import glob
import math
from functools import reduce
from time import time

import random
from collections import Counter
from razdel import tokenize
from typing import Any, Dict, List, Optional, Tuple, Union
import unittest

import numpy as np
import pandas as pd
import torch
from torch.optim import AdamW
from torch import nn
import torch.nn.functional as F
from mlm_pytorch.mlm_pytorch import MLM
from torch.utils.data import DataLoader
from tqdm import trange
from transformers import get_linear_schedule_with_warmup
from transformers.utils import logging
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import pipeline

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset

In [4]:
from evaluate import load

In [5]:
# from probing.classifier import MLP, LogReg, MDLLinearModel
# from probing.data_former import TextFormer
# from probing.encoder import TransformersLoader
# from probing.metric import Metric
# from probing.utils import KL_Loss, ProbingLog, lang_category_extraction, save_log

# logging.set_verbosity_warning()
# logger = logging.get_logger("probing")

# Портим текст

In [6]:
class TextIlliteracy:
    """Class for tokenizing text and changing variant of grammatical category
       in words of chosen parts of speech"""
    _tokens: List[str]

    def __init__(self, text: str) -> None:
        """Initializes an object"""
        self._text = text
        self._tokens = []
    
    def get_original_text(self) -> str:
        return self._text

    def tokenize_text(self) -> List[str]:
        """Tokenizes text"""
        pass

    def spoil_text(self, gram: str, postag_list: List[str]) -> str:
        """Changes grammatical markers to random inside the choosen category
           in all words of choosen parts of speech,
           if this grammatical category is relevant for such POS"""
        pass

In [7]:
class TextIlliteracyRus(TextIlliteracy):
    """Class tokenize russian text and changes variant of grammatical category
       in words of chosen parts of speech."""
    
    _tokens: List[str]
    # https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs
    
    __postags = ["NOUN", "ADJF", "ADJS", "COMP", "VERB", "INFN", "PRTF", "PRTS",
                 "GRND", "NUMR", "ADVB", "NPRO", "PRED", "PREP", "CONJ", "PRCL", "INTJ"]
    
    __grams = {"nmbr": ["sing", "plur"],
               "case": ["nomn", "gent", "datv", "accs", "ablt",
                        "loct", "voct", "gen2", "acc2", "loc2"],
               "anim": ["anim", "inan"],
               "gndr": ["masc", "femn", "neut", "ms-f"],
               "aspc": ["perf", "impf"],
               "trns": ["tran", "intr"],
               "pers": ["1per", "2per", "3per"],
               "tens": ["pres", "past", "futr"],
               "mood": ["indc", "impr"],
               "invl": ["incl", "excl"],
               "voic": ["actv", "pssv"]}

    def tokenize_text(self) -> List[str]:
        """For russian:
           tokenizes text"""
        if self._tokens != []:
            tokens = self._tokens
        else:
            tokens_with_boundaries = list(tokenize(self._text))
            # получили список токенов с границами
            tokens = [] # список токенов с пробелами в нужных местах
            prev_tok_end = 0
            for substring in tokens_with_boundaries:
                if substring.start != prev_tok_end:
                    tokens.append(" ")
                tokens.append(substring.text)
                prev_tok_end = substring.stop
            self._tokens = tokens
        return tokens
        
    def spoil_text(self, gram: str="nmbr", postag_list: List[str]=__postags) -> str:
        """For russian:
           changes grammatical markers to random inside the choosen category
           in all words of choosen parts of speech,
           if this grammatical category is relevant for such POS"""
        # берёт список частей речи и категорию,
        # которую у этих частей речи надо портить рандомными вариантами
        
        if self._tokens == []:
            self._tokens = self.tokenize_text()

        tokens = self._tokens

        changed_list = []
        for tok in tokens:
            tok_analysed = morph.parse(tok)[0]
            if tok_analysed.tag.POS in postag_list:
                new_gram_val = random.choice(TextIlliteracyRus.__grams[gram])
                if tok_analysed.inflect({new_gram_val}) is not None:
                    changed_tok = tok_analysed.inflect({new_gram_val}).word
                    if tok[0].isupper():
                        # чтобы при изменении буква оставалась заглавной в т.ч. для слова "Ма́трица"
                        changed_tok = changed_tok[0].upper()+changed_tok[1:]
                    changed_list.append(changed_tok)
                else:
                    changed_list.append(tok)
            else:
                changed_list.append(tok)
        changed_text = "".join(changed_list)
        return changed_text

In [8]:
# для других языков порча текста может основываться на других специализированных библиотеках
# в том числе на трансдьюсерах
# class TextIlliteracyEng, например

## Тестирование

(смотреть удобнее тут, но .py файл тоже есть.)

In [9]:
text = """Ма́трица — математический объект, записываемый в виде прямоугольной таблицы элементов кольца или поля (например, целых, действительных или комплексных чисел), который представляет собой совокупность строк и столбцов, на пересечении которых находятся его элементы. Количество строк и столбцов задает размер матрицы. Хотя исторически рассматривались, например, треугольные матрицы[1], в настоящее время говорят исключительно о матрицах прямоугольной формы, так как они являются наиболее удобными и общими.
Матрицы широко применяются в математике для компактной записи систем линейных алгебраических или дифференциальных уравнений. В этом случае количество строк матрицы соответствует числу уравнений, а количество столбцов — количеству неизвестных. В результате решение систем линейных уравнений сводится к операциям над матрицами."""
tokens = ['Ма́трица', ' ', '—', ' ', 'математический', ' ', 'объект', ',', ' ', 'записываемый', ' ', 'в', ' ', 'виде',
          ' ', 'прямоугольной', ' ', 'таблицы', ' ', 'элементов', ' ', 'кольца', ' ', 'или', ' ', 'поля', ' ', '(',
          'например', ',', ' ', 'целых', ',', ' ', 'действительных', ' ', 'или', ' ', 'комплексных', ' ', 'чисел', ')',
          ',', ' ', 'который', ' ', 'представляет', ' ', 'собой', ' ', 'совокупность', ' ', 'строк', ' ', 'и', ' ',
          'столбцов', ',', ' ', 'на', ' ', 'пересечении', ' ', 'которых', ' ', 'находятся', ' ', 'его', ' ', 'элементы',
          '.', ' ', 'Количество', ' ', 'строк', ' ', 'и', ' ', 'столбцов', ' ', 'задает', ' ', 'размер', ' ', 'матрицы',
          '.', ' ', 'Хотя', ' ', 'исторически', ' ', 'рассматривались', ',', ' ', 'например', ',', ' ', 'треугольные', ' ',
          'матрицы', '[', '1', ']', ',', ' ', 'в', ' ', 'настоящее', ' ', 'время', ' ', 'говорят', ' ', 'исключительно',
          ' ', 'о', ' ', 'матрицах', ' ', 'прямоугольной', ' ', 'формы', ',', ' ', 'так', ' ', 'как', ' ', 'они', ' ',
          'являются', ' ', 'наиболее', ' ', 'удобными', ' ', 'и', ' ', 'общими', '.', ' ', 'Матрицы', ' ', 'широко',
          ' ', 'применяются', ' ', 'в', ' ', 'математике', ' ', 'для', ' ', 'компактной', ' ', 'записи', ' ', 'систем',
          ' ', 'линейных', ' ', 'алгебраических', ' ', 'или', ' ', 'дифференциальных', ' ', 'уравнений', '.', ' ', 'В',
          ' ', 'этом', ' ', 'случае', ' ', 'количество', ' ', 'строк', ' ', 'матрицы', ' ', 'соответствует', ' ', 'числу',
          ' ', 'уравнений', ',', ' ', 'а', ' ', 'количество', ' ', 'столбцов', ' ', '—', ' ', 'количеству', ' ',
          'неизвестных', '.', ' ', 'В', ' ', 'результате', ' ', 'решение', ' ', 'систем', ' ', 'линейных', ' ',
          'уравнений', ' ', 'сводится', ' ', 'к', ' ', 'операциям', ' ', 'над', ' ', 'матрицами', '.']

In [10]:
class TestTextIlliteracyRus(unittest.TestCase):

    def setUp(self):
        print("setting up an object")
        self.textilliteracyrus = TextIlliteracyRus(text)
        
    def test_get_original_text(self):
        print("testing get_original_text")
        self.assertEqual(self.textilliteracyrus.get_original_text(), text)

    def test_tokenize_text(self):
        print("testing tokenize_text")
        self.assertEqual(self.textilliteracyrus.tokenize_text(), tokens)
    
    def test_spoil_text(self):
        print("testing spoil_text")
        self.assertIsInstance(self.textilliteracyrus.spoil_text(), str)
        self.assertIsInstance(self.textilliteracyrus.spoil_text("tens"), str)
        self.assertIsInstance(self.textilliteracyrus.spoil_text("case", ["NOUN", "ADJF"]), str)
        self.assertIsInstance(self.textilliteracyrus.spoil_text("tens", ["NOUN", "ADJF"]), str)
        
    def test_hidden(self):
        # print("testing hidden _text")
        # with self.assertRaises(AttributeError):
        #     self.textilliteracyrus._text
        # print("testing hidden _tokens")
        # with self.assertRaises(AttributeError):
        #     self.textilliteracyrus._tokens
        print("testing hidden __postags")
        with self.assertRaises(AttributeError):
            TextIlliteracyRus.__postags
        print("testing hidden __grams")
        with self.assertRaises(AttributeError):
            TextIlliteracyRus.__grams 

In [80]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_get_illiterate_UDtexts (__main__.TestIlliterateUD) ... 

setting up an object
testing get_illiterate_UDtexts


ok
test_get_original_text (__main__.TestTextIlliteracyRus) ... ok
test_hidden (__main__.TestTextIlliteracyRus) ... ok
test_spoil_text (__main__.TestTextIlliteracyRus) ... 

setting up an object
testing get_original_text
setting up an object
testing hidden __postags
testing hidden __grams
setting up an object
testing spoil_text


ok
test_tokenize_text (__main__.TestTextIlliteracyRus) ... 

setting up an object
testing tokenize_text


ok

----------------------------------------------------------------------
Ran 5 tests in 5.495s

OK


## Примеры работы класса

In [12]:
a = TextIlliteracyRus(text)
a.get_original_text()

'Ма́трица — математический объект, записываемый в виде прямоугольной таблицы элементов кольца или поля (например, целых, действительных или комплексных чисел), который представляет собой совокупность строк и столбцов, на пересечении которых находятся его элементы. Количество строк и столбцов задает размер матрицы. Хотя исторически рассматривались, например, треугольные матрицы[1], в настоящее время говорят исключительно о матрицах прямоугольной формы, так как они являются наиболее удобными и общими.\nМатрицы широко применяются в математике для компактной записи систем линейных алгебраических или дифференциальных уравнений. В этом случае количество строк матрицы соответствует числу уравнений, а количество столбцов — количеству неизвестных. В результате решение систем линейных уравнений сводится к операциям над матрицами.'

In [13]:
# при неосторожности __postags и __grams не поменяются
print(TextIlliteracyRus.__postags)
TextIlliteracyRus.__postags = ['NOUN', 'ADJF','ADJS']
print(TextIlliteracyRus.__postags)

AttributeError: type object 'TextIlliteracyRus' has no attribute '__postags'

In [ ]:
a.tokenize_text()

In [14]:
a.spoil_text("case", ["NOUN", "ADJF"])

'Ма́трице — математическим объекта, записываемый в вида прямоугольной таблицу элементов кольца или полем (например, целых, действительные или комплексных числам), которым представляет собой совокупности строки и столбцы, на пересечения которых находятся его элементы. Количества строки и столбцами задает размер матрице. Хотя исторически рассматривались, например, треугольных матрицы[1], в настоящего временем говорят исключительно о матрицах прямоугольной формы, так как они являются наиболее удобные и общим. Матрице широко применяются в математика для компактной записи систем линейными алгебраических или дифференциальным уравнения. В этом случая количество строками матрице соответствует числом уравнениям, а количестве столбцам — количество неизвестные. В результатом решение системам линейные уравнениях сводится к операции над матрицы.'

In [15]:
# Если у POS нет такой грам.категории текст просто не меняется.
# Можно было бы ограничить, чтоб на нерелевантной паре прогамма бы ломалась,
# но мой вариант мне больше нравится (это спорно, можно думать)
a.spoil_text("tens", ["NOUN"])

'Ма́трица — математический объект, записываемый в виде прямоугольной таблицы элементов кольца или поля (например, целых, действительных или комплексных чисел), который представляет собой совокупность строк и столбцов, на пересечении которых находятся его элементы. Количество строк и столбцов задает размер матрицы. Хотя исторически рассматривались, например, треугольные матрицы[1], в настоящее время говорят исключительно о матрицах прямоугольной формы, так как они являются наиболее удобными и общими. Матрицы широко применяются в математике для компактной записи систем линейных алгебраических или дифференциальных уравнений. В этом случае количество строк матрицы соответствует числу уравнений, а количество столбцов — количеству неизвестных. В результате решение систем линейных уравнений сводится к операциям над матрицами.'

In [16]:
a.spoil_text("tens")

'Ма́трица — математический объект, записывавший в виде прямоугольной таблицы элементов кольца или поля (например, целых, действительных или комплексных чисел), который представляет собой совокупность строк и столбцов, на пересечении которых находятся его элементы. Количество строк и столбцов задает размер матрицы. Хотя исторически рассматривались, например, треугольные матрицы[1], в настоящее время говорят исключительно о матрицах прямоугольной формы, так как они являются наиболее удобными и общими. Матрицы широко применяются в математике для компактной записи систем линейных алгебраических или дифференциальных уравнений. В этом случае количество строк матрицы соответствует числу уравнений, а количество столбцов — количеству неизвестных. В результате решение систем линейных уравнений сводился к операциям над матрицами.'

# Получаем неправильные предложения для выбранных языков, для выбранных грамматических категорий в папки файлов

In [17]:
class IlliterateUD:
    __lang_textilliteracy = {"rus": TextIlliteracyRus}
    def get_illiterate_UDtexts(self, gramcats: List[str]=["nmbr"],
                               lang_list: List[str]=["rus"]) -> None:
        """Creates a folder with good sentences from UD for each language"""
        for lang in lang_list:
            UD_filepath = "UD_sentences/"+lang+"_UD_sentences.csv"
            dirname = lang+"_for_perplexity"
            if not os.path.exists(dirname): os.makedirs(dirname)
            TextIlliteracyLang = IlliterateUD.__lang_textilliteracy[lang]
            gramcat_spoiled_sent = {}
            for gramcat in gramcats:
                gramcat_spoiled_sent[gramcat] = []

            with open(UD_filepath, 'r') as f:
                sents = f.readlines()
                notspoiledUD_filepath = dirname+"/"+lang+".csv"
                with open(notspoiledUD_filepath, 'w') as f1:
                    f1.write("".join(sents[:100]))

                for sent in sents[:100]:
                    sent_obj = TextIlliteracyLang(sent.strip())
                    for gramcat in gramcats:
                        spoiled_sent = sent_obj.spoil_text(gramcat)
                        gramcat_spoiled_sent[gramcat].append(spoiled_sent)
            for gramcat in gramcats:
                spoiledUD_filepath = dirname+"/"+lang+"_spoiledUD_"+gramcat+".csv"
                with open(spoiledUD_filepath, 'w') as f2:
                    f2.write("\n".join(gramcat_spoiled_sent[gramcat]))

In [18]:
b = IlliterateUD()
b.get_illiterate_UDtexts(["nmbr", "case", "pers", "tens"])

In [79]:
class TestIlliterateUD(unittest.TestCase):

    def setUp(self):
        print("setting up an object")
        self.illiterateUD = IlliterateUD()
        
    def test_get_illiterate_UDtexts(self, gramcats: List[str]=["nmbr"],
                                          lang_list: List[str]=["rus"]) -> None:
        print("testing get_illiterate_UDtexts")
        self.assertEqual(self.illiterateUD.get_illiterate_UDtexts(), None)
        self.assertEqual(self.illiterateUD.get_illiterate_UDtexts(["tens", "pers", "case"]), None)

# Будущие классы для полной структуры

In [19]:
class AIRI_Probing:
    pass

Здесь можно посмотреть Probing_framework:

https://github.com/AIRI-Institute/Probing_framework/blob/46f7ebad3af63b2b3b07fbe99398ce3eb27ed2b7/probing/pipeline.py#L24

https://github.com/AIRI-Institute/Probing_framework/blob/main/scripts/probing_experiment.py

In [52]:
class ConlluUDParser:
    """Creates folders with sentences from UD for each language
       for chosen type of probing"""
       # Надо переписать ConlluUDParser в фреймворке,
       # чтобы можно было добывать данные для их задачи и моей задачи
    def __init__(self) -> None:
        pass
    def get_good_UDtexts(self, lang_list: List[str]=["rus"]) -> None:
        """Creates a folder with good sentences from UD for each language
           for perplexity probing"""
        # Здесь для каждого языка сохранются все предложения в папку UD_sentences
        # Так русского в этой папке лежит rus_UD_sentences.csv
        pass

In [58]:
d = ConlluUDParser()
d.get_good_UDtexts(["rus"])

# Перплексия
(чем меньше, тем лучше)

In [74]:
class PerplexityProbing(AIRI_Probing):
    #lang_model_name = {"rus": "distilbert-base-multilingual-cased"}

    def __init__(self, gramcats: List[str]=["nmbr"],
                 lang_list: List[str]=["rus"]) -> None:
        """Initializes an object"""
        # Это же фасад?
        self.__gramcats = gramcats
        self.__lang_list = lang_list
        self.conllu = ConlluUDParser()
        self.spoiled_texts = IlliterateUD()

    def get_perplexity_results(self) -> Dict[str, Dict[str, float]]:
        """Gets languages and grammatical categories to check"""

        conllu = self.conllu
        conllu.get_good_UDtexts(self.__lang_list)
        getting_texts = self.spoiled_texts
        getting_texts.get_illiterate_UDtexts(self.__gramcats, self.__lang_list)

        lang_perplexityscores = {}
        for lang in self.__lang_list:
            print(lang)
            #model_name = self.lang_model_name[lang]
            model_name = "distilbert-base-multilingual-cased"
            model = AutoModelForMaskedLM.from_pretrained(model_name)
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            block_size = 128
            
            def tokenize_function(examples):
                  return tokenizer(examples["text"])
            def group_texts(examples):
                    # Concatenate all texts.
                    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
                    total_length = len(concatenated_examples[list(examples.keys())[0]])
                    # We drop the small remainder, we could add padding
                    # if the model supported it instead of this drop,
                    # you can customize this part to your needs.
                    total_length = (total_length // block_size) * block_size
                    # Split by chunks of max_len.
                    result = {k: [t[i : i + block_size] for i in range(0,
                                                                       total_length,
                                                                       block_size)]
                              for k, t in concatenated_examples.items()}
                    result["labels"] = result["input_ids"].copy()
                    return result
            
            training_args = TrainingArguments(model_name,
                                  evaluation_strategy = "epoch",
                                  learning_rate=2e-5,
                                  weight_decay=0.01)
            filename_perplexity = Counter()
            for filename in glob.glob(lang+"_for_perplexity/*.csv"):
                if filename[33:-4] in self.__gramcats or filename == lang+"_for_perplexity/"+lang+".csv":
                    # берём файл, если он про нужную категорию или если он с правильными предложениями
                    print(filename)
                    datasets = load_dataset("text",
                                            data_files={"validation": filename})
                    tokenized_datasets = datasets.map(tokenize_function,
                                                      batched=True,
                                                      num_proc=4,
                                                      remove_columns=["text"])
    
                    lm_datasets = tokenized_datasets.map(group_texts,
                                             batched=True,
                                             batch_size=1000,
                                             num_proc=4)
                    trainer = Trainer(model=model,
                                      args=training_args,
                                      eval_dataset=lm_datasets["validation"])
                    eval_results = trainer.evaluate()
        
                    filename_perplexity[filename] = math.exp(eval_results['eval_loss'])
                    print(filename_perplexity.most_common())
                    lang_perplexityscores[lang] = filename_perplexity.most_common()

            return lang_perplexityscores

In [75]:
c = PerplexityProbing(["nmbr", "case", "pers", "tens"], ["rus"])
c.get_perplexity_results()

rus


loading configuration file config.json from cache at /Users/tbkazakova/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/fb240273126596a03b35c85793d2e82a5b13ac79/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-multilingual-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 119547
}

loading weights file pytorch_model.bin from cache at /Users/tbkazakova/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/fb240273126596a03b35c85793d2e82a5b13ac79/pytorch_model.bin
All 

rus_for_perplexity/rus_spoiledUD_case.csv


Using custom data configuration default-007280e195132a0b


Extracting data files: 100%|██████████████████████| 1/1 [00:00<00:00, 80.06it/s]


...

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 107.30it/s]


Dataset text downloaded and prepared to /Users/tbkazakova/.cache/huggingface/datasets/text/default-1659fa0d21832e86/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


#0: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00, 36.00ba/s]

#1:   0%|                                                 | 0/1 [00:00<?, ?ba/s]

#2: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00, 30.77ba/s]



#0: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00, 25.09ba/s]


#2:   0%|                                                 | 0/1 [00:00<?, ?ba/s]


#3: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00, 24.77ba/s]
***** Running Evaluation *****
  Num examples = 24
  Batch size = 8


[('rus_for_perplexity/rus_spoiledUD_case.csv', 3.9193330434320828), ('rus_for_perplexity/rus_spoiledUD_nmbr.csv', 3.887017576653393), ('rus_for_perplexity/rus_spoiledUD_tens.csv', 3.845178846973014), ('rus_for_perplexity/rus.csv', 3.73473068311676), ('rus_for_perplexity/rus_spoiledUD_pers.csv', 3.629800816860565)]


{'rus': [('rus_for_perplexity/rus_spoiledUD_case.csv', 3.9193330434320828),
  ('rus_for_perplexity/rus_spoiledUD_nmbr.csv', 3.887017576653393),
  ('rus_for_perplexity/rus_spoiledUD_tens.csv', 3.845178846973014),
  ('rus_for_perplexity/rus.csv', 3.73473068311676),
  ('rus_for_perplexity/rus_spoiledUD_pers.csv', 3.629800816860565)]}

У нас система:

- ConlluUDParser - умеет переводить conllu файлы в разные csv форматы (для перрплексии - get_good_UDtexts(список_языков) - пока в разработке
- AIRI_Probing - общий класс для всех экспериментов AIRI - пока в разработке
- PerplexityProbing(AIRI_Probing) - наш эксперимент про перплексию
    - TextIlliteracy - родительский класс для порчи текстов
    - TextIlliteracyLang(TextIlliteracy) - лингвоспецифичные класс для порчи текстов
    - IlliterateUD - сохраняет нужные испорченные тексты в нужные папки и файлики


Мы:
- Выбираем тип пробинга: перплексия (а есть другие в т.ч. то, что у Олега в pipeline)
- Даём список список языков, список моделей для каждого языка, список категорий, которые мы хотим посмотреть
- Получаем словарь, язык-модель: ранжированный по важности список категорий (сильнее ухудшается перплексия - важнее) с цифрой перплексии

Внутри PerplexityProbing:

- для каждого языка из UD достаём тексты и сохраняем в папке UD_sentences файл lang_UD_sentences.csv списки предложений (by ConlluUDParser)
- для каждой грамматической категории получаем lang_spoiledUD_gramcat.csv со списками предложений, испорченными по категориям, кладём в папки lang_for_perplexity (by IlliterateUD, который использует TextIlliteracyLang(TextIlliteracy))
- для каждой пары язык-модель считаем перплексию для каждого из наборов предложений
- ранжируем грамматические категории по важности внутри модели-языка, получаем на выходе словарь

# Теоретические вопросы к происходящему

1. Надо менять категорию во всех местах? На рандом или неправильную (рандом из неправильных?)
- у меня просто рандом (можно обсуждать)

2. Как потом сравнивать удивление от изменений в категории, если род прилагательных и время у глагола имеют разную частотность в тексте?
Как нормализовывать? 30 слов: 10 глаголов - удивление/10\*30, 4 прилагательных - удивление/4\*30 Так?
- контролировать кол-во слов с изменениями? - спорно
- пока никак не нормализую, подумать, обсудить

3. Мы теряем информацию об абзацах (пробелоподобных символах - не пробелах), заменяя их пробелами. Ок?
- ок, потому что мы смотрим на предложения, а не длинные тексты

4. Если у POS нет такой грам.категории текст просто не меняется. Можно было бы ограничить, чтоб на нерелевантной паре прогамма бы ломалась или хотя бы писала, что пользователь делает что-то странное.
- не хочу, чтоб пограмма ломалась, но предупреждение, да, можно выдавать

5. Логичнее же смотреть на предложения, а не тексты? Вроде так.